In [1]:
import xgb_test

### Happy Path XGBoost Model

In [2]:
data = xgb_test.get_data()
X_train, X_test, y_train, y_test = xgb_test.split_data(data)
model = xgb_test.fit_model(X_train, y_train)
preds, rmse = xgb_test.score_model(model, X_test, y_test)
print("Predictions: " + str(preds))
print("RMSE: %f" % (rmse))

[17:04:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Predictions: [ 8.531183  17.426376  24.354538  10.202603  17.86703   20.111704
 17.825245   7.0048833 12.371497  18.518875  18.143133  14.239707
 11.409273  17.309526  13.857576  14.114464  13.485846  26.225672
 13.496281  10.878549  11.365604  12.844106  18.621658  22.8889
 22.134113  13.578582  10.041029  14.692133  14.988752  13.096905
 13.522167  19.658325   7.2066092 14.5406    15.21958   22.480894
 16.636026  10.085008  12.709627  23.140484  16.660902  13.496281
  8.958481  24.830044  12.231392  17.061684  13.437938  13.822018
 12.229067  10.875528  19.658325  16.804878  13.622031   7.8421807
 14.484597  10.775787  11.339119   7.332822  19.603703   7.9418693
 12.913391  13.190836  11.924444  11.435618  14.4757595 15.929399
 16.288265  12.295227  15.335289  18.830666  13.095968  15.788654
 12.999216  17.24653   10.544081  10.580927   8.360582  13.825303
 1

### Convert XGBoost Model to ONNX

In [3]:
import onnx_test

In [4]:
booster = model.get_booster()
features = booster.feature_names

In [5]:
onnx_model_path = "model_files/test_xgb.onnx"
onnx_model = onnx_test.convert_xgb_to_onnx(model, len(features), onnx_model_path)

The maximum opset needed by this model is only 1.


In [6]:
onnx_pred = onnx_test.score_onnx_model(onnx_model_path, X_test)

### Compare ONNX results and XGB results

In [7]:
for i in range(0,len(preds)):
    print(preds[i]-onnx_pred[i])

[9.536743e-07]
[0.]
[0.]
[0.]
[-1.9073486e-06]
[1.9073486e-06]
[0.]
[0.]
[0.]
[0.]
[0.]
[9.536743e-07]
[-9.536743e-07]
[1.9073486e-06]
[0.]
[9.536743e-07]
[0.]
[1.9073486e-06]
[0.]
[0.]
[0.]
[-9.536743e-07]
[0.]
[3.8146973e-06]
[0.]
[9.536743e-07]
[0.]
[0.]
[0.]
[9.536743e-07]
[0.]
[0.]
[4.7683716e-07]
[9.536743e-07]
[9.536743e-07]
[1.9073486e-06]
[1.9073486e-06]
[0.]
[9.536743e-07]
[0.]
[0.]
[0.]
[0.]
[1.9073486e-06]
[-9.536743e-07]
[3.8146973e-06]
[0.]
[9.536743e-07]
[9.536743e-07]
[9.536743e-07]
[0.]
[0.]
[0.]
[4.7683716e-07]
[0.]
[0.]
[0.]
[0.]
[0.]
[4.7683716e-07]
[0.]
[0.]
[-9.536743e-07]
[-9.536743e-07]
[9.536743e-07]
[1.9073486e-06]
[0.]
[9.536743e-07]
[9.536743e-07]
[-3.8146973e-06]
[9.536743e-07]
[9.536743e-07]
[0.]
[1.9073486e-06]
[0.]
[0.]
[9.536743e-07]
[0.]
[0.]
[0.]
[0.]
[4.7683716e-07]
[1.9073486e-06]
[9.536743e-07]
[9.536743e-07]
[1.9073486e-06]
[0.]
[0.]
[9.536743e-07]
[-3.8146973e-06]
[9.536743e-07]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[9.536743e-07]
[0.]
[9.536743e-07

### Create a dask XGB Model

In [8]:
import dask_xgb_test
from dask.distributed import Client

In [9]:
client = Client()  # connect to cluster
client

Client Scheduler: tcp://127.0.0.1:42687 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 32 Memory: 1.03 TB


In [10]:
# Get Dask Dataframe
dask_data = dask_xgb_test.get_dask_data(data)
# Get Feature List (Removed PRICE, because that is the target)
features = dask_data.drop('PRICE', axis=1)
# Split data to get training set
X_train_dask, y_train_dask = dask_xgb_test.split_data(dask_data, features.columns, 'PRICE')
# Fit Dask XGB model
dask_model = dask_xgb_test.fit_model(X_train_dask, y_train_dask).get_booster()

### Predict dask XGB model

In [11]:
predictions = dask_xgb_test.score_model(client, dask_model, X_train_dask)

### Convert Dask XGB model to ONNX

In [12]:
onnx_model_path = "model_files/test_dask_xgb.onnx"
features_dask = dask_model.feature_names
onnx_model = onnx_test.convert_xgb_to_onnx(dask_model, len(features_dask), onnx_model_path)

ValueError: No proper operator name found for '<class 'xgboost.core.Booster'>'